In [32]:

# pip install sentence-transformers neo4j pandas scikit-learn pydantic requests

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from pydantic import BaseModel
from typing import List, Optional
import pandas as pd
import numpy as np
from neo4j import GraphDatabase
import requests
import ollama


In [20]:
NEO4J_URI = "neo4j+s://7d3d4dbd.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "4cQimVVQmSVbF5rh6BA-qjWJcIQkmRnyjKXMoxco0Pw"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

def test_connection():
    try:
        with driver.session() as session:
            result = session.run("RETURN 1")
            return result.single()[0]
    except Exception as e:
        return f"Erreur de connexion : {str(e)}"
    finally:
        driver.close()

print(test_connection())


1


In [21]:
class Book(BaseModel):
    ISBN: str
    title: str
    description: Optional[str] = ""
    author: Optional[str] = ""

class Recommendation(BaseModel):
    isbn: str
    title: str
    score: float
    author: str

In [22]:
def load_books():
    with driver.session() as session:
        query = """
        MATCH (b:Book)
        RETURN b.ISBN AS ISBN, b.title AS title, b.author AS author
        """
        result = session.run(query)
        return [Book(**record) for record in result]

def load_ratings():
    with driver.session() as session:
        query = """
        MATCH (u:User)-[r:RATED]->(b:Book)
        RETURN u["UserId"] AS user_id, b.ISBN AS book_id, toInteger(r.rating) AS rating
        """
        result = session.run(query)
        return pd.DataFrame([record.data() for record in result])


In [23]:
def create_user_similarity_matrix(ratings_df):
    user_item_matrix = ratings_df.pivot_table(index='user_id', columns='book_id', values='rating', aggfunc='mean').fillna(0)
    cosine_sim = cosine_similarity(user_item_matrix)
    print("Matrice de similarité calculée.")
    return user_item_matrix, cosine_sim


In [24]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

def get_book_embeddings(books: List[Book]):
    descriptions = [book.title or "" for book in books]
    embeddings = embedder.encode(descriptions, convert_to_tensor=True)
    return {book.ISBN: emb for book, emb in zip(books, embeddings)}


In [25]:
def recommend_books(user_id, user_item_matrix, cosine_sim_users, book_embeddings, books_metadata, top_n=5):
    if user_id not in user_item_matrix.index:
        print("Utilisateur non trouvé.")
        return []

    user_idx = user_item_matrix.index.get_loc(user_id)
    sim_scores = list(enumerate(cosine_sim_users[user_idx]))
    top_similar_users = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:6]

    user_rated_books = set(user_item_matrix.columns[user_item_matrix.loc[user_id] > 0])
    score_sum, similarity_sum = {}, {}

    for neighbor_idx, similarity in top_similar_users:
        neighbor_id = user_item_matrix.index[neighbor_idx]
        neighbor_ratings = user_item_matrix.loc[neighbor_id]

        for book, rating in neighbor_ratings.items():
            if rating > 0 and book not in user_rated_books:
                score_sum[book] = score_sum.get(book, 0) + rating * similarity
                similarity_sum[book] = similarity_sum.get(book, 0) + similarity

    predicted_ratings = {
        book: score_sum[book] / similarity_sum[book]
        for book in score_sum if similarity_sum[book] != 0
    }

    for book_id in predicted_ratings:
        if book_id in book_embeddings:
            target_embedding = book_embeddings.get(book_id)
            rated_embeddings = [book_embeddings[isbn] for isbn in user_rated_books if isbn in book_embeddings]

            if rated_embeddings:
                avg_user_embedding = np.mean(rated_embeddings, axis=0)
                content_similarity = cosine_similarity([avg_user_embedding], [target_embedding])[0][0]
                predicted_ratings[book_id] *= (0.7 + 0.3 * content_similarity)

    recommended_books = sorted(predicted_ratings.items(), key=lambda x: x[1], reverse=True)[:top_n]
    # return [Recommendation(isbn=isbn, title=books_metadata.get(isbn, "Titre inconnu"), score=score) for isbn, score in recommended_books]
    return [
        Recommendation(
            isbn=isbn,
            title=books_metadata.get(isbn, ("Titre inconnu", ""))[0],
            author=books_metadata.get(isbn, ("", "Auteur inconnu"))[1],
            score=score
        )
        for isbn, score in recommended_books
    ]

In [26]:
# Chargement des données
ratings_df = load_ratings()
books = load_books()
books_dict = {book.ISBN: (book.title, book.author) for book in books}

# Matrices utilisateur-livre & similarité
user_item_matrix, cosine_sim_users = create_user_similarity_matrix(ratings_df)

# Embeddings
book_embeddings = get_book_embeddings(books)


C:\Users\dell latitude 9510\AppData\Local\Temp\ipykernel_5220\628689800.py:11: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\dell latitude 9510\AppData\Local\Temp\ipykernel_5220\628689800.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


Matrice de similarité calculée.


In [29]:
def generate_summary_with_ollama(title: str, author: str):
    prompt = f"Peux-tu me faire un résumé clair et concis du livre suivant ?\nTitre: {title}\nAuteur: {author}"
    response = ollama.chat(model="mistral:latest", messages=[{"role": "user", "content": prompt}])
    return response['message']['content']

In [ ]:
# Trouver un user qui a noté assez de livres
# user_rating_counts = ratings_df.groupby('user_id')['book_id'].count()
# active_users = user_rating_counts[user_rating_counts >= 5].index.tolist()
# print("Exemples d'user_id actifs :", active_users[:10])

# Tester avec un user
user_id = 165  

# Recommandation
recs = recommend_books(user_id, user_item_matrix, cosine_sim_users, book_embeddings, books_dict, top_n=5)

# Affichage des recommandations
print(f"\nRecommandations pour l'utilisateur {user_id} :")

for rec in recs:
    print(f"\n📘 {rec.title} (ISBN: {rec.isbn}, Auteur: {rec.author}) — Score: {rec.score:.2f}")
    desc = next((b.title for b in books if b.ISBN == rec.isbn), "")
    print(generate_summary_with_ollama(rec.title, rec.author))


📘 Angels &amp; Demons (ISBN: 0671027360, Auteur: Dan Brown) — Score: 7.79
Résumé :  Titre : Les anges et les démons
Auteur : Dan Brown

Résumé clair et concis : Dans un voyage à travers la ville de Rome, le symbologiste Robert Langdon est invité à résoudre un mystérieux meurtre qui a impliqué l'Ordre des Templiers. En recherchant les preuves pour déterminer qui est derrière les crimes, Langdon découvre un complot diabolique connu sous le nom de la Pièce d'or et de la main du Pape, une formule de transmutation de l'or en poussière d'or, inventée par Albert Einstein. En utilisant son expertise sur les symboles cachés dans la religion catholique, Langdon tente de résoudre le mystère avant que les anges et les démons ne s'affrontent pour la souveraineté du monde.

📘 The Alibi (ISBN: 0446608653, Auteur: Sandra Brown) — Score: 6.97
Résumé :  Livre : The Alibi
   Auteur : Sandra Brown

Résumé :

Dans le livre The Alibi, l'auteur Sandra Brown raconte l'histoire de l'avocat Taylor MacFee qui e